### Imports

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from io import StringIO
import json
import os, sys
from tqdm import tqdm
import re

sys.path.append("../")
from pprint import pprint

from scripts.parser_epic import EpicParser

### epic

In [2]:
parser = EpicParser()
records = []
failed = 0
json_dir = os.path.join("../", "data", "med_records_8500_json")
for json_file in tqdm(os.listdir(json_dir)):
    path_json = os.path.join(json_dir, json_file)

    # read json
    with open(path_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    try:

        patient_id = data["{urn:hl7-org:v3}recordTarget"][
            "{urn:hl7-org:v3}patientRole"
        ]["{urn:hl7-org:v3}id"][0]["extension"]

        # passport_date = data["{urn:hl7-org:v3}recordTarget"]["{urn:hl7-org:v3}patientRole"][
        #     "{urn:hl7-ru:identity}IdentityDoc"
        # ]["{urn:hl7-ru:identity}IssueDate"]["value"]

        data_in_record = {
            "patient_id": patient_id,
            # "passport_date": passport_date,
            "record": json_file.removesuffix(".json"),
            "gender": parser.get_sex(data),
            "birthday": parser.get_age(data),
            "amnez_life": parser.get_amnez_life(data),
            "amnez_disease": parser.get_amnez_d(data),
            "condition": parser.get_condition(data),
            # TODO: make clearer names
            "ward_table": parser.get_ward_table(data).to_dict(),
            "full_wards": parser.compute_full_wards(data),
            "final_table1": parser.get_final_table1(data).to_dict(),
            "final_table2": parser.get_final_table2(data).to_dict(),
        }

        diagnosis_code_and_desc = parser.get_features_from_diagnosis_table(data)
        data_in_record.update(diagnosis_code_and_desc)

        # add date of entering ward
        table = parser.get_ward_table(data)
        data_in_record["Дата поступления"] = table.loc[
            table["Название"] == "Приемное отделение",
            "Дата поступления",
        ].values[0]
        # data_in_record["Дата поступления"] = None

        a, b, c = parser.get_gosp_info(data)  # a same as diagnosis_table
        data_in_record.update(
            {
                "gospitalisation_type": b,
                "gospitalisation_delivery": c,
            }
        )

        records.append(data_in_record)
    except:
        failed += 1

records_in_dataframe = pd.DataFrame(records)
print(f"failed to parse {failed} files")

100%|██████████| 8518/8518 [01:22<00:00, 103.14it/s]


failed to parse 12 files


In [3]:
records_in_dataframe["patient_id"].unique().shape

(7280,)

In [7]:
records_in_dataframe.head(3)

,patient_id,record,gender,birthday,amnez_life,amnez_disease,condition,ward_table,full_wards,final_table1,...,secondary_disease_desc,complication_of_main_disease_code,main_disease_code,secondary_disease_code,Дата поступления,gospitalisation_type,gospitalisation_delivery,background_disease_desc,background_disease_code,Конкурирующее заболевание
0,75DB9BE5-92BE-432F-AA97-2CC5A56B8BBC,EMD_EPIC_DISCHARGE_185171131_185171131,Мужской,20000101,ОБЩЕБИОГРАФИЧЕСКИЕ СВЕДЕНИЯ НА СЕВЕРЕ ДАВНО \...,"АНАМНЕЗ ИБС С 2016Г., КОГДА ПЕРЕНЕС ОСТРЫЙ ИМ ...","[Состояние: Средней тяжести, Жалобы: НА МОМЕНТ...","{'Название': {0: 'Приемное отделение', 1: 'Кар...",{'Сведения о пребывании пациента в Приемное от...,{'Характер основного заболевания': {0: 'Острое...,...,"Сахарный диабет 2 типа, инсулиннезависимый. Це...",[I50.9],[I20.0],"[I10, E11.8, E66.9]",03.04.2023 12:00,Первичный,Плановая госпитализация,NaN,NaN,NaN
1,55218D18-4928-6138-E055-000000000001,EMD_EPIC_DISCHARGE_145291569_145291569,Мужской,20000101,АНАМНЕЗ ЖИЗНИ:,АНАМНЕЗ ЗАБОЛЕВАНИЯ ИБС С 1986 ГОДА КОГДА ВПЕ...,"[Состояние: Удовлетворительное, Жалобы: ЖАЛОБЫ...","{'Название': {0: 'Приемное отделение', 1: 'Кар...",{'Сведения о пребывании пациента в Приемное от...,{'Характер основного заболевания': {0: 'Острое...,...,Хронический гепатит В без дельта-агента минима...,"[I48.0, I50.1]",[I21.0],"[I69.4, B18.0, I85.9]",19.01.2019 09:46,Первичный,Доставлен бригадой скорой помощи,"Гипертоническая болезнь III стадии, Артериальн...",[I11.9],NaN
2,E85181B7-58BC-4879-9181-B758BCF879C2,EMD_EPIC_DISCHARGE_167695907_167695907,Женский,20000101,АНАМНЕЗ ЖИЗНИ:,АНАМНЕЗ ЗАБОЛЕВАНИЯ ИБС РАНЕЕ ОТРИЦАЕТ. С 16....,"[Состояние: Удовлетворительное, Жалобы: ЖАЛОБЫ...","{'Название': {0: 'Приемное отделение', 1: 'Кар...",{'Сведения о пребывании пациента в Приемное от...,{'Характер основного заболевания': {0: 'Острое...,...,Ожирение 3 степени (ИМТ 46 кг/м2),[I50.1],[I20.0],[E66.9],22.05.2021 17:40,Первичный,Доставлен бригадой скорой помощи,"Сахарный диабет 2 типа, уровень гликогемоглоби...","[E11.8, I11.9]",NaN


In [6]:
dir_path = "../data/parsed_dataframes"
os.makedirs(dir_path, exist_ok=True)
file_name = "epic_records_8500.parquet"
file_path = os.path.join(dir_path, file_name)
records_in_dataframe.astype(str).to_parquet(
    file_path,
    engine="pyarrow",
)

### old table

In [56]:
parser = EpicParser()
records = []
json_dir = os.path.join("../", "data", "json_records", "DocsNew1")
for json_file in os.listdir(json_dir):
    path_json = os.path.join(json_dir, json_file)

    # read json
    with open(path_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    data_in_record = {
        "record": json_file.removesuffix(".json"),
        "gender": parser.get_sex(data),
        "birthday": parser.get_age(data),
        "amnez_life": parser.get_amnez_life(data),
        "amnez_disease": parser.get_amnez_d(data),
        "condition": parser.get_condition(data),
        # TODO: make clearer names
        "ward_table": parser.get_ward_table(data).to_dict(),
        "full_wards": parser.compute_full_wards(data),
        "final_table1": parser.get_final_table1(data).to_dict(),
        "final_table2": parser.get_final_table2(data).to_dict(),
    }
    diagnosis_code_and_desc = parser.get_features_from_diagnosis_table(data)
    data_in_record.update(diagnosis_code_and_desc)

    # add date of entering ward
    table = parser.get_ward_table(data)
    data_in_record["Дата поступления"] = table.loc[
        table["Название"] == "Приемное отделение",
        "Дата поступления",
    ].values[0]

    a, b, c = parser.get_gosp_info(data)  # a same as diagnosis_table
    data_in_record.update(
        {
            "gospitalisation_type": b,
            "gospitalisation_delivery": c,
        }
    )

    records.append(data_in_record)

records_in_dataframe = pd.DataFrame(records)

In [58]:
records_in_dataframe.head(3)

,record,gender,birthday,amnez_life,amnez_disease,condition,complication_of_main_disease_desc,main_disease_desc,secondary_disease_desc,complication_of_main_disease_code,main_disease_code,secondary_disease_code,gospitalisation_type,gospitalisation_delivery,background_disease_desc,background_disease_code
0,EMD_EPIC_DISCHARGE_182985535_182985535,Мужской,19710308,Б,"Считает себя больным с 13.01.2023 г, когда в 1...","[Состояние: Тяжелое, Жалобы: на пекуче-жгучую ...",ОСН 1 по Killip.,ИБС. Острый инфаркт миокарда нижней стенки ЛЖ ...,Ожирение 2 степени (ИМТ 36),[I50.1],[I21.4],[E66.9],Первичный,Плановая госпитализация,NaN,NaN
1,EMD_EPIC_DISCHARGE_186840484_186840484,Женский,19650620,"Эпидемиологический анамнез:Со слов пациента, с...",08.06.2023г около 20.30 в покое появились давя...,"[Состояние: Тяжелое, Жалобы: на давящие боли з...",ОСН I по Killip.,ИБС. Инфаркт миокарда с подъёмом сегмента ST п...,Ожирение 1 степени (ИМТ 32),[I50.1],[I21.0],[E66.9],Первичный,Доставлен бригадой скорой помощи,Гипертоническая болезнь III стадии. Неконтроли...,[I11.9]
2,EMD_EPIC_DISCHARGE_187352881_187352881,Мужской,19790326,"Эпидемиологический анамнез:Со слов пациента, с...","ранее ИБС отрицает, впервые стал отмечать колю...","[Состояние: Тяжелое, Жалобы: на давящие боли з...",ОСН I по Killip,ИБС. Острый инфаркт миокарда передне-перегород...,Ожирение 2 степени по ВОЗ(ИМТ 36 кг/м2),[I50.1],[I21.0],[E66.9],Повторный,Доставлен бригадой скорой помощи,NaN,NaN


In [9]:
dir_path = "../data/parsed_dataframes"
os.makedirs(dir_path, exist_ok=True)
file_name = "epic_records.parquet"
file_path = os.path.join(dir_path, file_name)
records_in_dataframe.to_parquet(file_path, engine="pyarrow")